In [361]:
import pandas as pd
import numpy as np
import geopandas as gpd
import locale
import babel.numbers
import decimal

In [350]:
## Set Options

In [351]:
locale.setlocale( locale.LC_ALL, '' )
'English_United States.1252'

'English_United States.1252'

In [275]:
trump_districts = pd.read_csv('https://docs.google.com/spreadsheets/d/11nBAu3jHlJe3N9bbmniawP_WuIs6FURl1u1qXjUddZQ/pub?gid=2080606191&single=true&output=csv')

In [276]:
len(trump_districts)

436

In [277]:
trump_districts.head(3)
#trump_districts.dtypes

,state,district,min_num,min_amt,split_num,split_amt,max_num,max_amt
0,MS,1,5,4700,5,4700,5,4700
1,KS,3,13,7110,14,7735,15,8360
2,WA,5,4,2750,4,2750,4,2750


In [278]:
trump_districts['split_num'] = trump_districts['split_num'].astype(int)
trump_districts['district'] = trump_districts['district'].astype(int).map("{:02}".format)
trump_districts.head(3)

,state,district,min_num,min_amt,split_num,split_amt,max_num,max_amt
0,MS,01,5,4700,5,4700,5,4700
1,KS,03,13,7110,14,7735,15,8360
2,WA,05,4,2750,4,2750,4,2750


In [279]:
#Convert district number to 'AL' for at larte districts - SEE NOTE BELOW
for i in al:
    trump_districts.loc[i, 'district'] = 'AL'
trump_districts.loc[al,:]

,state,district,min_num,min_amt,split_num,split_amt,max_num,max_amt
61,VI,AL,1,250,1,250,1,250
72,VT,AL,3,750,3,750,3,750
179,DE,AL,11,3598,11,3598,11,3598
183,WY,AL,10,5932,10,5932,10,5932
208,DC,AL,14,5654,14,5654,14,5654
291,SD,AL,6,3860,6,3860,6,3860
304,MT,AL,7,5043,7,5043,7,5043
349,AK,AL,15,11805,15,11805,15,11805
394,ND,AL,4,1984,4,1984,4,1984


In [280]:
trump_districts['cd_id'] = trump_districts.state.str.cat(trump_districts.district, sep='-')
trump_districts
len(trump_districts)

436

## Get district by 2012 winning party

In [281]:
party_by_district115C = pd.read_csv('https://docs.google.com/spreadsheets/d/1Motlc8nNUPOd-f_9G_lOW4fO073e1pdAX7V6DpBzkY8/pub?output=csv')

In [282]:
#note one district off. 
len(party_by_district115C)

435

In [283]:
party_by_district115C.head(1)
#party_by_district115C[party_by_district115C.CD.str.contains('ZZ')==True]

,CD,Incumbent,Party,Obama 2012,Romney 2012,Obama 2008,McCain 2008
0,AK-AL,"Young, Don",(R),41.2,55.3,38.1,59.7


In [284]:
#add variable for winnig party in 2012 election results for each district
party_by_district115C['2012_party'] = None
obama = party_by_district115C[party_by_district115C['Obama 2012'] >= party_by_district115C['Romney 2012']]

obama =obama.index
romney = party_by_district115C.index[~party_by_district115C.index.isin(obama)]

party_by_district115C.loc[obama,'2012_party'] = '(D)'
party_by_district115C.loc[romney,'2012_party'] = '(R)'

party_by_district115C['2012_party'].describe()

count     435
unique      2
top       (R)
freq      224
Name: 2012_party, dtype: object

In [285]:
dist_party = party_by_district115C[['CD', '2012_party']]
dist_party.columns = ['cd_id', 'party']
dist_party.head(3)

,cd_id,party
0,AK-AL,(R)
1,AL-01,(R)
2,AL-02,(R)


## get statefp codes 

In [391]:
#113th Congressional District Codes
#Source: http://www2.census.gov/geo/docs/reference/codes/files/national_cd113.txt
codes = pd.read_csv('https://docs.google.com/spreadsheets/d/1P7rSu7FCyhc4SU7EiCVDQoGC5n4DAbqnsgPcSXt3YKg/pub?gid=875229823&single=true&output=csv')

In [392]:
#isolate state codes
codes.head()
st_codes = codes[['STATE', 'STATEFP']]
st_codes.loc[:,'STATEFP'] = st_codes.loc[:,'STATEFP'].astype(int).map("{:02}".format)
st_codes.loc[:,'STATEFP'] = st_codes.loc[:,'STATEFP'].astype('str')
st_codes.head(3)

,STATE,STATEFP
0,AL,01
1,AL,01
2,AL,01


In [288]:
st_codes = st_codes.drop_duplicates()
len(st_codes)

56

## Clean Shape File

In [289]:
# Information on latest redistricting changes: http://redistricting.lls.edu/
#Census Bureau 113th congress congressional district shape file 
#Source: https://www.census.gov/cgi-bin/geo/shapefilesrd13/file-download
cds_113_shp = gpd.read_file('tl_rd13_us_cd113/tl_rd13_us_cd113.shp')

In [290]:
#cds_113_shp.info()

In [291]:
cds_113_shp.loc[:,'STATEFP'] = cds_113_shp.loc[:,'STATEFP'].astype(int).map("{:02}".format)
cds_113_shp.loc[:,'STATEFP'] = cds_113_shp.loc[:,'STATEFP'].astype('str')

In [293]:
cds_113_shp.sort_values(by='STATEFP').head(1)

,ALAND,AWATER,CD113FP,CDSESSN,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,MTFCC,NAMELSAD,STATEFP,geometry
172,26304372148,330254983,07,113,N,0107,+32.4492349,-087.6405014,C2,G5200,Congressional District 7,01,"POLYGON ((-88.47322699999999 31.893856, -88.47..."


In [295]:
cds_orig_shape = cds_113_shp.copy()
type(cds_orig_shape.geometry)

geopandas.geoseries.GeoSeries

In [296]:
#map state codes to state abbreviations 
cds_113_shp = pd.merge(cds_113_shp, st_codes, on = 'STATEFP', how = 'left')
cds_113_shp.head(1)

,ALAND,AWATER,CD113FP,CDSESSN,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,MTFCC,NAMELSAD,STATEFP,geometry,STATE
0,14227693111,414257512,01,113,N,3701,+36.1119104,-077.5009008,C2,G5200,Congressional District 1,37,"POLYGON ((-78.95837499999999 36.001372, -78.95...",NC


In [297]:
len(cds_113_shp)

444

In [298]:
type(cds_113_shp.geometry)

pandas.core.series.Series

In [299]:
#check how at-large districts are handled. 
cds_113_shp[cds_113_shp['STATEFP'] == '66']

,ALAND,AWATER,CD113FP,CDSESSN,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,MTFCC,NAMELSAD,STATEFP,geometry,STATE
379,543389293,934520040,98,113,N,6698,+13.4382959,+144.7729279,C4,G5200,Delegate District (at Large),66,"POLYGON ((144.563426 13.448065, 144.563574 13....",GU


In [300]:
#Align cd naming convention with other files. 
#strip text
cds_113_shp['NAMELSAD'] = cds_113_shp['NAMELSAD'].str.strip("Congressional District ")

In [302]:
#rename at-large states 'AL'
at_large = cds_113_shp[~cds_113_shp['NAMELSAD'].str.isdigit()]
at_large_states = at_large['STATE']
at_large_states

93     MP
112    WY
123    VT
134    PR
136    AS
138    AK
173    VI
199    CT
207    ND
213    DC
237    DE
269    MT
283    SD
294    IL
337    MI
379    GU
Name: STATE, dtype: object

In [303]:
#convert district label in at-large states to 'AL' 
at_large= at_large.index
cds_113_shp.loc[at_large,'NAMELSAD'] = 'AL'

In [304]:
#convert district numbers to 2-digit format
digits = cds_113_shp[cds_113_shp['NAMELSAD'].str.isdigit()]
digits = digits.index
cds_113_shp.loc[digits,'NAMELSAD'] = cds_113_shp.loc[digits,'NAMELSAD'].astype(int).map("{:02}".format)

In [306]:
#concat STATE and district number (NAMELSAD) to create district ID 
cds_113_shp['cd_id'] = cds_113_shp.STATE.str.cat(cds_113_shp.NAMELSAD, sep='-')
cds_113_shp.head(3)

,ALAND,AWATER,CD113FP,CDSESSN,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,MTFCC,NAMELSAD,STATEFP,geometry,STATE,cd_id
0,14227693111,414257512,01,113,N,3701,+36.1119104,-077.5009008,C2,G5200,01,37,"POLYGON ((-78.95837499999999 36.001372, -78.95...",NC,NC-01
1,20229273757,10851171398,03,113,N,3703,+35.4057865,-076.6296737,C2,G5200,03,37,"POLYGON ((-78.27217399999999 34.538072, -78.27...",NC,NC-03
2,1730858364,86473790,04,113,N,2504,+41.9971989,-071.2275271,C2,G5200,04,25,"POLYGON ((-71.60164399999999 42.219664, -71.59...",MA,MA-04


In [307]:
cds_shp = cds_113_shp.copy()
len(cds_shp)

444

## Check for miscoded or missing values in cd_id between datasets 

In [308]:
#check that there are no values in Koos data not in cds_shp
s1 = dist_party['cd_id']
s2 = cds_shp['cd_id']

check = s1[~s1.isin(s2)]

In [309]:
len(check)

0

In [310]:
#check that there are no values in trump_districts data not in cds_shp
s1 = trump_districts['cd_id']
s2 = cds_shp['cd_id']

check = s1[~s1.isin(s2)]

In [311]:
len(check)

0

In [312]:
#initial pass showed 9 at-large districts coded differently
check

Series([], Name: cd_id, dtype: object)

In [313]:
#use index values from check to recode at large districts in trump_districts above. 
al = check.index
al

Int64Index([], dtype='int64')

In [314]:
s1 = trump_districts['cd_id']
s2 = dist_party['cd_id']

check = s1[~s1.isin(s2)]

In [315]:
check
#two at large districts in trump data and not in dist_party data. 
#DC = democratic 
#VI = democratic 

61     VI-AL
208    DC-AL
Name: cd_id, dtype: object

In [316]:
dist_party.cd_id.describe()

count       435
unique      435
top       MA-09
freq          1
Name: cd_id, dtype: object

In [317]:
trump_districts.cd_id.describe()

count       436
unique      436
top       MA-09
freq          1
Name: cd_id, dtype: object

In [318]:
#check that there isn't type or other error
check1 = dist_party[dist_party['cd_id'].str.contains('VI') == True]
check2 = dist_party[dist_party['cd_id'].str.contains('DC') == True]
print (check1)
print (check2)

Empty DataFrame
Columns: [cd_id, party]
Index: []
Empty DataFrame
Columns: [cd_id, party]
Index: []


In [319]:
s1 = dist_party['cd_id']
s2 = trump_districts['cd_id']

check = s1[~s1.isin(s2)]

In [320]:
# only 1 disrict is not in the trump dataset, likely because he didn't receive a donation from this district
check

285    NY-15
Name: cd_id, dtype: object

In [321]:
new_rows = pd.DataFrame(data = {'cd_id':['VI-AL','DC-AL'], 'party':['(D)', '(D)']})
new_rows

,cd_id,party
0,VI-AL,(D)
1,DC-AL,(D)


In [322]:
dist_party = dist_party.append(new_rows)

In [323]:
dist_party.describe()

,cd_id,party
count,437,437
unique,437,2
top,MA-09,(R)
freq,1,224


In [324]:
dist_table = dist_party.groupby('party').size()
dist_table

party
(D)    213
(R)    224
dtype: int64

## Add party affiliation to shape file

In [325]:
# merge party affiliation with shape file, with an 'inner' merge, 
#thereby removing extraneous large districts from shape file

cds_shp2 = pd.merge(cds_shp, dist_party, on = 'cd_id', how = 'inner')
cds_shp2.head(2)

,ALAND,AWATER,CD113FP,CDSESSN,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,MTFCC,NAMELSAD,STATEFP,geometry,STATE,cd_id,party
0,14227693111,414257512,01,113,N,3701,+36.1119104,-077.5009008,C2,G5200,01,37,"POLYGON ((-78.95837499999999 36.001372, -78.95...",NC,NC-01,(D)
1,20229273757,10851171398,03,113,N,3703,+35.4057865,-076.6296737,C2,G5200,03,37,"POLYGON ((-78.27217399999999 34.538072, -78.27...",NC,NC-03,(R)


In [327]:
type(cds_shp2.geometry)

pandas.core.series.Series

In [330]:
#remake GeoSeries
crs = None
geometry = cds_shp2['geometry']
cds_shp2 = gpd.GeoDataFrame(cds_shp2, crs=crs, geometry=geometry)

In [331]:
type(cds_shp2.geometry)

geopandas.geoseries.GeoSeries

In [334]:
#cds_shp2.to_file('cds_113c_byparty2.shp', driver="ESRI Shapefile")

## Add geom centroids to Trump contributions data 

In [335]:
district_centroids = cds_shp2.centroid
district_centroids.head(3)

0    POINT (-77.51369409502156 36.09876447243866)
1    POINT (-76.61938684826103 35.41056159586817)
2    POINT (-71.23234410807179 41.99125362437578)
dtype: object

In [336]:
type(district_centroids)

geopandas.geoseries.GeoSeries

In [337]:
len(district_centroids)

437

In [376]:
#add geom points to trump data
trump_pts = pd.DataFrame({'cd_id': cds_shp2.cd_id, 'party': cds_shp2.party, 'geometry': district_centroids})

In [377]:
trump_pts.head(3)

,cd_id,geometry,party
0,NC-01,POINT (-77.51369409502156 36.09876447243866),(D)
1,NC-03,POINT (-76.61938684826103 35.41056159586817),(R)
2,MA-04,POINT (-71.23234410807179 41.99125362437578),(D)


In [378]:
trump_pts = pd.merge(trump_pts, trump_districts, on = 'cd_id', how = 'inner')
trump_pts.head(2)

,cd_id,geometry,party,state,district,min_num,min_amt,split_num,split_amt,max_num,max_amt
0,NC-01,POINT (-77.51369409502156 36.09876447243866),(D),NC,01,3,706,6,2331,9,3956
1,NC-03,POINT (-76.61938684826103 35.41056159586817),(R),NC,03,5,1515,6,2265,8,3015


In [379]:
trump_pts_split = trump_pts[['cd_id', 'geometry', 'party', 'state', 'district', 'split_num', 'split_amt']]
trump_pts_split.shape

(436, 7)

In [380]:
trump_pts_split.head(2)

,cd_id,geometry,party,state,district,split_num,split_amt
0,NC-01,POINT (-77.51369409502156 36.09876447243866),(D),NC,01,6,2331
1,NC-03,POINT (-76.61938684826103 35.41056159586817),(R),NC,03,6,2265


In [383]:
trump_pts_split.loc[:,'dollar_amt'] = trump_pts_split.loc[:,'split_amt'].astype(int).map('${:,}'.format)
trump_pts_split.head(2)

,cd_id,geometry,party,state,district,split_num,split_amt,dollar_amt
0,NC-01,POINT (-77.51369409502156 36.09876447243866),(D),NC,01,6,2331,"$2,331"
1,NC-03,POINT (-76.61938684826103 35.41056159586817),(R),NC,03,6,2265,"$2,265"


In [388]:
#create display text var 
trump_pts_split['text'] = trump_pts_split[['cd_id','split_num', 'dollar_amt']].apply(lambda x : '<strong>District:</strong> {}<br><strong>Number of Contributors:</strong> {}<br><strong>Total Amount:</strong> {}'.format(x[0],x[1], x[2]), axis=1)
trump_pts_split

/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,cd_id,geometry,party,state,district,split_num,split_amt,dollar_amt,text
0,NC-01,POINT (-77.51369409502156 36.09876447243866),(D),NC,01,6,2331.000000,"$2,331",<strong>District:</strong> NC-01<br><strong>Nu...
1,NC-03,POINT (-76.61938684826103 35.41056159586817),(R),NC,03,6,2265.000000,"$2,265",<strong>District:</strong> NC-03<br><strong>Nu...
2,MA-04,POINT (-71.23234410807179 41.99125362437578),(D),MA,04,28,14837.333330,"$14,837",<strong>District:</strong> MA-04<br><strong>Nu...
3,MA-09,POINT (-70.48147673169255 41.69417780735502),(D),MA,09,13,5899.000000,"$5,899",<strong>District:</strong> MA-09<br><strong>Nu...
4,MA-08,POINT (-70.94364106832272 42.19298731657016),(D),MA,08,15,175855.333300,"$175,855",<strong>District:</strong> MA-08<br><strong>Nu...
5,VA-11,POINT (-77.294793779834 38.78810996655279),(D),VA,11,6,1978.166667,"$1,978",<strong>District:</strong> VA-11<br><strong>Nu...
6,CO-04,POINT (-103.3752189589195 39.06218571170709),(R),CO,04,13,9345.333333,"$9,345",<strong>District:</strong> CO-04<br><strong>Nu...
7,KY-06,POINT (-84.16266380335531 38.04833852400073),(R),KY,06,9,4482.000000,"$4,482",<strong>District:</strong> KY-06<br><strong>Nu...
8,AL-06,POINT (-86.71298133896737 33.26337958140939),(R),AL,06,6,3165.333333,"$3,165",<strong>District:</strong> AL-06<br><strong>Nu...
9,TX-04,POINT (-95.40442140966132 33.31037079071876),(R),TX,04,8,4895.000000,"$4,895",<strong>District:</strong> TX-04<br><strong>Nu...


In [389]:
#remake GeoSeries
crs = None
geometry = trump_pts_split['geometry']
trump_data = gpd.GeoDataFrame(trump_pts_split, crs=crs, geometry=geometry)

In [390]:
trump_data.to_file('trump_data.shp', driver="ESRI Shapefile")